# FF Convergence

In [1]:
import sys
sys.path.insert(0, '../../../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import tensorflow as tf
import random
import pickle as pkl

from qiskit.quantum_info import Operator, Statevector, random_unitary
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm
from math import ceil

from quantum_tools import circuit_to_matrix, partial_trace
from kraus_channels import isomery_to_kraus, KrausMap
from quantum_channel import ConcatChannel, DilutedChannel, IdentityChannel, ChoiMapStatic
from utils import numberToBase, kron
from spectrum import complex_spacing_ratio, channel_spectrum, Spectrum, mean_spacing, coat_spectrum
from analysis import spectrum_to_angular, spectrum_to_radial, hist_ensamble
from qiskit.circuit.library import iSwapGate, XGate, YGate, HGate, CXGate, RGate, RZGate
from quantum_circuits import integrable_circuit, nonintegrable_circuit, haar_random
from synthetic_data import generate_map_data, generate_spam_data, generate_spam_benchmark
from optimization import ModelQuantumMap, ModelSPAM, Logger, model_saver
from loss_functions import ProbabilityMSE, ProbabilityRValue, channel_fidelity_loss
from spam import SPAM, InitialState, POVMwQR as POVM, CorruptionMatrix

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=1)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


## Circuits

In [2]:
def sample_traceout_csr(n, n_anc, reps, circuit_type, L):
    channel_list = []
    spectra_list = []
    csr_list = []


    for _ in tqdm(range(reps)):
        circuit = circuit_type(n+n_anc, L)
        U = Operator(circuit).data

        kraus_map = isomery_to_kraus(U, rank = 2**n_anc)
        channel_list.append(kraus_map)
        
        spectrum = channel_spectrum(kraus_map)
        spectra_list.append(spectrum)
        csr = complex_spacing_ratio(spectrum, log=False)
        csr_list.append(csr)

    return  channel_list, spectra_list, csr_list

In [9]:
def fit_spam(inputs, 
             targets,
             num_iter = 3000,
             verbose = False):
    d = targets.shape[1]
    spam_model = SPAM(init = InitialState(d),
                    povm = CorruptionMatrix(d),
                    )

    spam_opt = ModelSPAM(spam_model, tf.keras.optimizers.Adam(learning_rate=0.01))
        
    spam_opt.pretrain(100, verbose=False)

    spam_opt.train(inputs = inputs,
                    targets = targets,
                    num_iter = num_iter,
                    verbose = verbose,
                )
    
    return spam_model
    

def fit_model(inputs, 
              targets,
              channel, 
              spam_model,
              num_iter = 3000,
              verbose=False):
    d = targets.shape[1]
    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                        rank = d**2,
                                        spam = spam_model,
                                        ),
                    loss_function = ProbabilityMSE(),
                    optimizer = tf.optimizers.Adam(learning_rate=0.01),
                    logger = Logger(loss_function_list = [ProbabilityRValue(), channel_fidelity_loss], sample_freq=100),
                )
    model.train(inputs = inputs, 
                targets = targets,
                inputs_val = [inputs, None],
                targets_val = [targets, [channel]],
                N=500,
                num_iter = num_iter,
                verbose = verbose,)
    
    return model

## Free Fermion

### Generate Benchmarks

In [10]:
n = 4
d = 2**n

[channel_FF_list, spectra_FF_list, csr_FF_list] =  pkl.load(open("data/FF_synthetic_benchmark.pkl", "rb"))

### Generate Synthetic Data and Fit

In [11]:
model_list = []

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

for channel in channel_FF_list:
    
    spam_target = generate_spam_benchmark(n=4, c1=0.95, c2=0.95)
    inputs_spam, targets_spam = generate_spam_data(spam_target, shots=12000)
    
    spam_model = fit_spam(inputs_spam,
                        targets_spam,
                        num_iter = 3000,
                        verbose = False)
        

    inputs_map, targets_map = generate_map_data(channel, 
                                                spam_target = spam_target,
                                                N_map=5000-6**n, 
                                                shots = 12000)

    model = fit_model(inputs_map,
                    targets_map,
                    channel,
                    spam_model,
                    num_iter = 3000,
                    verbose = False
                    )

    model_list.append(model)

model_saver(model_list, "model_list_FF_noisy_moreSamples.model")


[0.9981521922919037, -0.9884585385416366]
[0.9986147894359223, -0.9886104750098561]
[0.9983940653951896, -0.9893165003686872]
[0.9982619097040094, -0.9885639419281108]
[0.9983583274605841, -0.9898139723142935]
[0.998446015247062, -0.9901642522198657]
[0.9985741167476936, -0.9891778772749799]
[0.9983513774975263, -0.9900525935043739]
[0.9984239765098771, -0.9903400241300618]
[0.9985073968948348, -0.990274111410653]


## Chaotic Haar Random

### Generate Benchmarks

In [12]:
n = 4
d = 2**n

[channel_FF_list, spectra_FF_list, csr_FF_list] =  pkl.load(open("data/chaotic_synthetic_benchmark.pkl", "rb"))

### Generate Synthetic Data and Fit

In [13]:
model_list = []

np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

for channel in tqdm(channel_FF_list):
    
    spam_target = generate_spam_benchmark(n=4, c1=0.95, c2=0.95)
    inputs_spam, targets_spam = generate_spam_data(spam_target, shots=12000)
    
    spam_model = fit_spam(inputs_spam,
                        targets_spam,
                        num_iter = 3000,
                        verbose = False)
        

    inputs_map, targets_map = generate_map_data(channel, 
                                                spam_target = spam_target,
                                                N_map=5000-6**n, 
                                                shots = 12000)

    model = fit_model(inputs_map,
                    targets_map,
                    channel,
                    spam_model,
                    num_iter = 3000,
                    verbose = False
                    )

    model_list.append(model)

model_saver(model_list, "model_list_chaotic_noisy_moreSamples.model")


  0%|          | 0/10 [00:00<?, ?it/s]

[0.9965744651865197, -0.9879946079271656]
[0.9965611043539115, -0.988970383709137]
[0.9965775501547759, -0.9888708687755593]
[0.9966334601712581, -0.9886510992154597]
[0.9965908086503824, -0.9883169268835605]
[0.996595829316722, -0.9892094808837621]
[0.9965326888918862, -0.9884370116898662]
[0.9966119891249545, -0.9884342777160143]
[0.9966059946179475, -0.9890200276221915]
[0.9965333880773753, -0.9887756913523358]
